# SQL Database Work from Python and Pandas

Learning Objectives:
* Students will how to configure, start, and stop a PostgreSQL server on CoCalc.
* Students will learn how to create new databases, and how pandas DataFrame objects can be written to and read from database tables.
* Students will observe and form common SQL queries for selecting results and joining tables.

Video segments before class:
* Segments of [Mike Dane's freeCodeCamp.org SQL Tutorial](https://www.youtube.com/watch?v=HXV3zeQKqGY):
  * [SQL Basics: 43:31-52:25](https://youtu.be/HXV3zeQKqGY?t=2611)
  * [Basic Queries: 1:56:11-2:08:36](https://www.youtube.com/watch?v=HXV3zeQKqGY&t=6971s)

Before class:
* Read execute and study the notebook contents up to the "In Class" section.
* Make sure that you have correctly configured PostgreSQL and that the examples work in your virtual machine before class.

In class:
* We will work together on the exercises in section "In Class".

Homework after class:
* Complete the section labeled "Homework" below before the next class when it will be collected.


In [1]:
# Imports

import numpy as np
import pandas as pd
import psycopg2
from psycopg2 import sql
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from sqlalchemy import create_engine


# PostgreSQL

[SQL (Structured Query Language)](https://en.wikipedia.org/wiki/SQL), pronounced "**s**e**q**ue**l**", refers to a family of similar relational database query languages.  Think of a database as a collection of persistent tables in memory allowing concurrent, distributed access, querying, and modification.  Indeed, you have already had a hint of what it is like to work with databases, as DataFrames, with their concat, append, merge, join, aggregation, and grouping operations, are similar in many ways similar to database tables.  Even the names of DataFrame operations, like inner and outer join, originate from relational database operations.

So why is it important to learn about database interactions as a Data Scientist?  The simple answer is that databases are the primary source of large structured information in business.  It would be common for Data Scientists to draw their data from database sources.  It is also often more efficient to perform queries on the database side (using some SQL variant) and only create DataFrames with data of interest, as opposed to loading entire massive, complex tables into DataFrames and filtering or processing the data afterwards.

[PostgreSQL](https://www.postgresql.org/), a.k.a. "Postgres", is one of the most popular free and [open source](https://en.wikipedia.org/wiki/Open_source) relational database management systems.  It also happens to be installed in your CoCalc virtual machine.  In this lesson, we will not attempt to teach the contents of a database _course_.  Rather thing of this as a starting point of reference for initial exposure with encouragement and resources for further learning.

SQL-type languages, and query languages in general, are far less expressive than general-purpose programming languages.  They are focused on efficient manipulation of tabular data.

## CoCalc PostgreSQL Configuration and Workflow

Configuration of PostgreSQL only happens once.  Follow the instructions of ["Step 1: Setup"](https://doc.cocalc.com/howto/postgresql.html#step-1-setup) in ["PostgreSQL Server"](https://doc.cocalc.com/howto/postgresql.html) only once. This will involve creating a Linux terminal window and copy-pasting the given setup commands.

Before a session when you use PostgreSQL, you need to enter the first Step 2 command from this terminal window:

After a session when you use PostgreSQL, you need to enter the Step 4 command from this terminal window:

We will connect to the database via Python, so you will primarily use the terminal window to start and stop the database server.  Try both the start and stop commands above in your terminal window.  Then notice that you can find and enter a previous command _without re-entering it_ by using the up and down arrow keys to move to previous and next commands, respectively, in your command history.  You can simply hit enter to enter the command; your cursor need not be at the end of the line.  Practice starting and stopping the PostgreSQL server a couple times using only arrow keys and enter.

Suffice it to say, it is worth [learning more about the ```bash``` shell](http://cs.gettysburg.edu/~tneller/dept/bash-tutorial.html) that runs in the terminal window as you have time beyond this course.  You can find many excellent free resources for ```bash``` learning at the bottom of our ["Department of Computer Science ```Bash``` Tutorial"](http://cs.gettysburg.edu/~tneller/dept/bash-tutorial.html).  (The tutorial itself is according to our department system file structure, not CoCalc's.  Even so, the tutorial can give you an idea of the most important ```bash``` commands to learn.)


## Connecting to the Database Server in Python

Our connection with the PostgreSQL database server will be facilitated by the ```psycopg2``` library, imported above.  Initially, PostgreSQL contains a few initial databases.  Below we create a connection object ```conn```.  One needs to specify a ```dbname``` database name to make a connection.  We can create other databases as well, but our ability to connect depends on there being a database with which to initially connect.  We connect to database "postgres" with the specified CoCalc local host.  We next create a cursor object ```cur``` which you can think of as a pointer to a database row when we execute a query.

Our first query, ```select datname from pg_database```, selects all database names in our PostgreSQL installation, and points to the first row of our results.  The ```fetchall``` returns all results as a list of row tuples.

In [2]:
conn = psycopg2.connect('dbname=postgres host=/home/user/postgres_data')
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
cur = conn.cursor()
cur.execute('drop database if exists all_work')
cur.execute('drop database if exists df_store')
cur.execute('select datname from pg_database')
rows = cur.fetchall()
print('Databases in our PostgreSQL installation:')
for row in rows:
    print('\t', row[0])


Databases in our PostgreSQL installation:
	 postgres
	 template1
	 template0


## Creating a Database

Having completed a connection to _a_ database, we can now create a new database.  We'll start by dropping (deleting) a database ```first_create``` if it already exists, and then we'll create it.

In [3]:
# Drop (delete) database 'first_create' if it exists
cur.execute('drop database if exists first_create')

# Create database 'first_create'
cur.execute('create database first_create')

# We now query to verify that our new database exists:
cur.execute('select datname from pg_database')
rows = cur.fetchall()
print('Databases in our PostgreSQL installation:')
for row in rows:
    print('\t', row[0])

# Finally, we drop (delete) the database:
cur.execute('drop database if exists first_create')


Databases in our PostgreSQL installation:
	 postgres
	 template1
	 template0
	 first_create


## From DataFrame to Database Table

DataFrames make use the ```sqlalchemy``` module to directly export a DataFrame to a database table using function [to_sql](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html). This module [supports PostgreSQL](https://docs.sqlalchemy.org/en/13/dialects/postgresql.html).

_Note: Pandas DataFrames have many ```to_``` prefix functions, converting directly to json, Excel, HTML, LaTeX, markdown, etc._ 

In [4]:
# From Jake VanderPlas PDSH, Ch. 3:
def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

# example DataFrame
df_test1 = make_df('ABC', range(3))

print('df_test1', df_test1, sep='\n')

try:
    cur.execute('create database df_store')
except Exception as ex:
    print(ex)

# See https://docs.sqlalchemy.org/en/13/dialects/postgresql.html#module-sqlalchemy.dialects.postgresql.psycopg2
# General Unix syntax: create_engine("postgresql+psycopg2://user:password@/dbname?host=/var/lib/postgresql")
engine = create_engine("postgresql+psycopg2://user:@/df_store?host=/home/user/postgres_data")
with engine.connect() as connection:
    try:
        df_test1.to_sql('df_test1', connection)  # name of table, sqlalchemy connection object
    except Exception as ex:
        print(ex)


df_test1
    A   B   C
0  A0  B0  C0
1  A1  B1  C1
2  A2  B2  C2


## From Database Query to DataFrame

Similarly, Pandas DataFrames can be directly created from a database table using function [read_sql_table](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql_table.html).

In [5]:
with engine.connect() as connection:
    try:
        df_test2 = pd.read_sql_table('df_test1', connection, index_col='index')  # name of table, sqlalchemy connection object
        df_test2.index.name = None
        print('df_test2', df_test2, sep='\n')
    except Exception as ex:
        print(ex)

try:
    cur.execute('drop database if exists df_store')
except Exception as ex:
    print(ex)

df_test2
    A   B   C
0  A0  B0  C0
1  A1  B1  C1
2  A2  B2  C2


## A Brief Demonstration of SQL

The following example database comes from

> Elmasri, Ramez. Shamkant Navathe. Fundamentals of Database Systems. 7th Edition. Pearson 2016. ISBN-10: 0-13-397077-0.

with example queries supplied by Prof. Clifton G.M. Presser.

We first create an example employee database if it doesn't already exist.

In [0]:
cur.execute('drop database if exists first_create')

sql_database_setup = """-- Create Employee database
CREATE TABLE EMPLOYEE
       (
         FNAME  VARCHAR(15) NOT NULL,
  	 MINIT  CHAR,
	 LNAME  VARCHAR(15) NOT NULL,
	 SSN  CHAR(9)  NOT NULL,
	 BDATE DATE,
  	 ADDRESS VARCHAR(30),
  	 SEX  CHAR,
  	 SALARY  DECIMAL(10,2),
  	 SUPERSSN CHAR(9),
  	 DNO  INT  NOT NULL,
  	 PRIMARY KEY(SSN)
       );

CREATE TABLE DEPARTMENT
       (
         DNAME  VARCHAR(15) NOT NULL,
  	 DNUMBER INT  NOT NULL,
  	 MGRSSN  CHAR(9)  NOT NULL,
  	 MGRSTARTDATE DATE,
  	 PRIMARY KEY(DNUMBER),
  	 UNIQUE(DNAME)
       );

CREATE TABLE DEPT_LOCATIONS
       (
         DNUMBER INT  NOT NULL,
  	 DLOCATION VARCHAR(15) NOT NULL,
  	 PRIMARY KEY(DNUMBER, DLOCATION)
       );

CREATE TABLE PROJECT
       (
         PNAME  VARCHAR(15) NOT NULL,
  	 PNUMBER INT  NOT NULL,
  	 PLOCATION VARCHAR(15),
  	 DNUM  INT  NOT NULL,
  	 PRIMARY KEY(PNUMBER),
  	 UNIQUE(PNAME)
       );

CREATE TABLE WORKS_ON
       (
         ESSN  CHAR(9)  NOT NULL,
  	 PNO  INT  NOT NULL,
  	 HOURS  DECIMAL(3,1),
  	 PRIMARY KEY(ESSN, PNO)
       );

CREATE TABLE DEPENDENT
       (
         ESSN  CHAR(9)  NOT NULL,
  	 DEPENDENT_NAME VARCHAR(15) NOT NULL,
  	 SEX  CHAR,
  	 BDATE  DATE,
  	 RELATIONSHIP VARCHAR(8),
  	 PRIMARY KEY(ESSN, DEPENDENT_NAME)
       );


-- fill tables
INSERT INTO DEPENDENT VALUES
('333445555','Alice','F','1986-04-05','DAUGHTER'),
('333445555','Theodore','M','1983-10-25','SON'),
('333445555','Joy','F','1958-05-03','SPOUSE'),
('987654321','Abner','M','1942-02-28','SPOUSE'),
('123456789','Michael','M','1988-01-04','SON'),
('123456789','Alice','F','1988-12-30','DAUGHTER'),
('123456789','Elizabeth','F','1967-05-05','SPOUSE');

INSERT INTO EMPLOYEE VALUES
('John','B','Smith','123456789','1965-01-09','731 Fondren, Houston, TX','M',30000,'333445555',5),
('Franklin','T','Wong','333445555','1955-12-08','638 Voss, Houston, TX','M',40000,'888665555',5),
('Alicia','J','Zelaya','999887777','1968-07-19','3321 Castle, Spring, TX','F',25000,'987654321',4),
('Jennifer','S','Wallace','987654321','1941-06-20','291 Berry,Bellaire,TX','F',43000,'888665555',4),
('Ramesh','K','Narayan','666884444','1962-09-15','975 Fire Oak, Humble, TX','M',38000,'333445555',5),
('Joyce','A','English','453453453','1972-07-31','5631 Rice, Houston, TX','F',25000,'333445555',5),
('Ahmad','V','Jabbar','987987987','1969-03-29','980 Dallas, Houston, TX','M',25000,'987654321',4),
('James','E','Borg','888665555','1937-11-10','450 Stone, Houston, TX','M',55000,'',1);

INSERT INTO DEPARTMENT VALUES
('Research',5,'333445555','1988-05-22'),
('Administration',4,'987654321','1995-01-01'),
('Headquarters',1,'888665555','1961-06-19');

INSERT INTO DEPT_LOCATIONS VALUES
(4,'Stafford'),
(5,'Bellaire'),
(5,'Sugarland'),
(5,'Houston');

INSERT INTO PROJECT VALUES
('ProductX',1,'Bellaire',5),
('ProductY',2,'Sugarland',5),
('ProductZ',3,'Houston',5),
('Computerization',10,'Stafford',4),
('Reorganization',20,'Houston',1),
('Newbenefits',30,'Stafford',4);

INSERT INTO WORKS_ON VALUES
('123456789',1,32.5),
('123456789',2,7.5),
('666884444',3,40.0),
('453453453',1,20.0),
('453453453',2,20.0),
('333445555',2,10.0),
('333445555',3,10.0),
('333445555',10,10.0),
('333445555',20,10.0),
('999887777',30,30.0),
('999887777',10,10.0),
('987654321',10,35.0),
('987654321',30,20.0),
('987654321',20,15.0),
('888665555',20,NULL);
"""

future_use = """
-- turn off constraints
-- MySql: SET foreign_key_checks = 0;
-- PostgreSQL:
ALTER TABLE EMPLOYEE DISABLE TRIGGER ALL;
ALTER TABLE DEPARTMENT DISABLE TRIGGER ALL;
ALTER TABLE DEPT_LOCATIONS DISABLE TRIGGER ALL;
ALTER TABLE PROJECT DISABLE TRIGGER ALL;
ALTER TABLE WORKS_ON DISABLE TRIGGER ALL;
ALTER TABLE DEPENDENT DISABLE TRIGGER ALL;

ALTER TABLE EMPLOYEE ADD FOREIGN KEY (SUPERSSN) REFERENCES EMPLOYEE(SSN);
ALTER TABLE EMPLOYEE ADD FOREIGN KEY (DNO) REFERENCES DEPARTMENT(DNUMBER);
ALTER TABLE DEPARTMENT ADD FOREIGN KEY (MGRSSN) REFERENCES EMPLOYEE(SSN);
ALTER TABLE DEPT_LOCATIONS ADD FOREIGN KEY (DNUMBER) REFERENCES DEPARTMENT(DNUMBER);
ALTER TABLE PROJECT ADD FOREIGN KEY (DNUM) REFERENCES DEPARTMENT(DNUMBER);
ALTER TABLE WORKS_ON ADD FOREIGN KEY (ESSN) REFERENCES EMPLOYEE(SSN);
ALTER TABLE WORKS_ON ADD FOREIGN KEY (PNO) REFERENCES PROJECT(PNUMBER);
ALTER TABLE DEPENDENT ADD FOREIGN KEY (ESSN) REFERENCES EMPLOYEE(SSN);

-- turn on constraints
-- MySql: SET foreign_key_checks = 1;
-- PostgreSQL:
ALTER TABLE EMPLOYEE ENABLE TRIGGER ALL;
ALTER TABLE DEPARTMENT ENABLE TRIGGER ALL;
ALTER TABLE DEPT_LOCATIONS ENABLE TRIGGER ALL;
ALTER TABLE PROJECT ENABLE TRIGGER ALL;
ALTER TABLE WORKS_ON ENABLE TRIGGER ALL;
ALTER TABLE DEPENDENT ENABLE TRIGGER ALL;
"""

conn_all_work = None
try:
    cur.execute('drop database if exists all_work') # Comment this line to avoid repeated creation
    cur.execute('create database all_work')
    # Reconnect to our new database
    conn_all_work = psycopg2.connect('dbname=all_work host=/home/user/postgres_data')
    with conn_all_work:
        cur_all_work = conn_all_work.cursor()
        try:
            cur_all_work.execute(sql_database_setup)
        except Exception as ex:
            print(ex)
except Exception as ex:
    print(ex)
finally:
    if conn_all_work:
        conn_all_work.close()  # Close the connection if it was created

### Example SQL queries

Finally, we will perform a few queries using ```psycopg2``` and have you explore a few more queries in class.  To better understand these queries, we present a "database schema" diagram of the Employee database tables with arrows representing related attributes across tables.

<img src="EmployeeSchema.png" alt="Employee Schema" title="Employee Schema" width="800" />

In [0]:
try:
    conn_all_work = psycopg2.connect('dbname=all_work host=/home/user/postgres_data')
    with conn_all_work:
        cur_all_work = conn_all_work.cursor()

        # Query 1: List all tables in the database
        cur_all_work.execute("""select table_name from information_schema.tables where table_schema = 'public'""")
        print('Employee database tables:')
        for table in cur_all_work.fetchall():
            print('\t', table[0])

        # Query 2: List all of the employees with salaries between $20,000 and $30,000.
        sql = 'select * from employee where salary > 20000 and salary < 30000;'
        cur_all_work.execute(sql)
        print('\n', sql)
        for employee in cur_all_work.fetchall():
            print(employee)

        # Query 3: Get just the first and last names from the 'employee' table
        sql = 'select fname as first, lname as last from employee as e;'
        cur_all_work.execute(sql)
        print('\n', sql)
        for employee in cur_all_work.fetchall():
            print(employee)

        # Query 4: What are all possible combinations of employees (first, last name) and departments.
        # (This is known as a "Cartesian product" (https://en.wikipedia.org/wiki/Cartesian_product).)
        sql = 'select fname, lname, dname department from employee e, department d;'
        cur_all_work.execute(sql)
        print('\n', sql)
        for employee in cur_all_work.fetchall():
            print(employee)

    conn_all_work.close()  # Close our connection to database all_work
except Exception as ex:
    print(ex)

# In Class

## More Sample SQL Queries

Using the all_work Employee database from above, perform the following different forms of "natural join" SQL queries and examine the results.

(1) ```select fname, lname, dname department from employee e, department d where e.dno=d.dnumber;```

(2) ```select fname, lname, dname from employee e, department d where e.dno=d.dnumber;```

(3) ```select fname, lname, dname from employee e join department d on e.dno=d.dnumber;```

Keywords: between ... and

(4) select fname, lname from employee where salary between 20000 and 30000;

Keywords: order by

(5) select fname, lname from employee order BY lname;

(6) select fname, lname from employee order BY salary desc;

(7) select fname, lname from employee order BY lname asc;

### SQL Joins

In the time remaining, create a new database called ```joins```.  Add to this database the example tables of the Wikipedia article [Join (SQL)](https://en.wikipedia.org/wiki/Join_(SQL)).  Perform the illustrative queries of various joins using these tables.

## Homework


**Exercise 1:** Complete the in-class exercise if you haven't already.

**Exercise 2:** The [SQL ```select``` statement](https://en.wikipedia.org/wiki/Select_(SQL)) is foundational to database queries.  Important keywords used include:

* ```from```, specifying the database table
* ```where```, specifying a filter condition on rows
* ```group by```, specifying a shared property of rows so that an aggregate function can be applied to each row group
* ```having```, specifying a filter among groups of the ```group by``` clause
* ```order by```, specifying how returned rows should be ordered, and
* ```as```, specifying a temporary alias for referring to tables or columns.

Using the ```all_work``` database, devise queries different from those above to illustrate each of these syntactic elements of SQL ```select``` statements.

(end of homework)